In [3]:
from harvest_data import *


In [2]:
conn, cur = make_db_connection()

In [3]:
update_channels(conn, cur)

[200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200]
[200]
[200, 200]
16 new videos discovered. Capturing metadata and adding to database.


In [4]:
update_videos(conn, cur)

current utc time is 2018-09-14 04:09:15
working in groups of 1000 records.
########group 1 of 1
[200, 200, 200]


In [38]:
now_f = datetime.datetime.strftime(datetime.datetime.utcnow(), "%Y-%m-%d %H:%M:%S")
print now_f 
cur.execute("SELECT video_id FROM public.videos_meta WHERE next_update < timestamp %(now)s ;", {"now":now_f})

2018-07-18 09:01:21


In [8]:
__, channel_timeseries = get_channel_data(channels)

[200]


In [6]:
cur.execute("SELECT video_id FROM public.videos_meta;")
channels = flatten(cur.fetchmany(size=30))

In [5]:
x = {"a":"A"}
print len(x)

1


In [35]:
example_date = datetime.datetime(2018,7,17,8,12,11)
print schedule(example_date)



2018-07-20 07:13:09.251000


In [5]:
cur.execute('''WITH ordered_timeseries as (
		select *
	 	from public.videos_timeseries
 		order by video_id, sample_time
	 ),
	 views_min_max as (
		 select video_id,
		 		first(sample_time) as sample_f,
		 		last(sample_time) as sample_l,
		 		first(view_count) as view_f,
		 		last(view_count) as view_l
		 from ordered_timeseries
		 group by video_id
	 )
SELECT
video_id,
view_l - view_f as delta_views,
(view_l - view_f)/(extract(epoch from (sample_l - sample_f))/86400) as views_per_day
from views_min_max
where extract(epoch from (sample_l - sample_f)) > 0
order by delta_views desc''')

In [6]:
yonivids_path = os.path.join(os.path.join(os.getcwd(),os.pardir),"fullVideosSep2018.txt")


In [48]:
import pandas as pd
header = ["meta","videoId"]
yonivids = pandas.read_csv(yonivids_path, names=header )
yonivids = yonivids.dropna()

In [45]:
yonividscol= yonivids.loc[:,"videoId"]
yonividscol.count()

23861

In [49]:
yonividslist= list(yonivids.loc[:,"videoId"])
yonividslist = map(lambda x: str(x).replace(" ",""), yonividslist)


In [52]:
yonivdata = get_video_data(yonividslist)

[200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200, 200,

In [59]:
yonivdatameta = yonivdata[0]

In [64]:
yonivdatameta[0]

{'channelId': u'UC4Xt-DUAapAtkfaWWkv4OAw',
 'description': u'An introduction to number systems, and how to convert between decimal, binary and hexadecimal numbers. \nAhora con subt\xedtulos en espa\xf1ol, escoge CC. Favor de dejarme feedback y correcciones en los comentarios abajo.',
 'duration': datetime.timedelta(0, 1007),
 'lastUpdate': datetime.datetime(2018, 9, 13, 7, 3, 50, 290000),
 'nextUpdate': datetime.datetime(2018, 9, 18, 7, 3, 50, 290000),
 'publishedAt': datetime.datetime(2014, 5, 14, 17, 13, 3),
 'tags': [u'binary',
  u'hexadecimal',
  u'decimal',
  u'number system',
  u'conversion',
  u'Binary Number',
  u'hex',
  u'binary to decimal'],
 'title': u'Number Systems - Converting Decimal, Binary and Hexadecimal',
 'videoId': u'aW3qCcH6Dao'}

In [63]:
upsert_video_meta(yonivdatameta, conn, cur)

In [72]:
for video_id in yonividslist:
    cur.execute("UPDATE videos_meta SET is_maths_content = (%s) where video_id = (%s);", (True, video_id))
conn.commit()

In [73]:
cur.execute("Select * from public.videos_meta where is_maths_content = true;")
vids = cur.fetchall()

In [75]:
db_vid_ids = [i[1] for i in vids]

In [87]:
print len(db_vid_ids)
print len(yonividslist)
print len(yonivdatameta)

23590
23861
23567


In [81]:
missed_vids = [i for i in yonividslist if i not in db_vid_ids]


In [88]:
get_video_data(missed_vids)

[200, 200, 200, 200, 200, 200]


([], [])

In [90]:
missed_vids[1]

'hCLfogkqzEk'